### Translate CODA ground truth to KITTIodometry (runtime: ~ 35min)

In [2]:
import matplotlib.pyplot as plt
import matplotlib.colors as col
from mpl_toolkits.mplot3d import Axes3D
import json
import os
import numpy as np
from tqdm import tqdm

#### Set paths
<div class="alert alert-block alert-warning">
<h4>ToDo:</h4>
<ol>
    <li> Set "coda_root" to the root of your <b>CODA directory</b>
    <li> Set "log_finn_root" to the root of your <b>inference directory</b>
</ol>
</div>

In [3]:
coda_root = f'/disk/ml/own_datasets/CODA'
log_finn_root = f'/disk/vanishing_data/ju878/log_finn/'

In [4]:
model_root = os.path.realpath('../../model_contradictions/')
create_coda_root = os.path.join(model_root, 'create_coda')

lidar_clustering_folder = os.path.join(coda_root, 'lidar_clustering')
lidar_labels_folder = os.path.join(coda_root, 'lidar_labels')
lidar_labels_annotation_folder = os.path.join(coda_root, 'lidar_labels_annotation')
annotation_validation = os.path.join(create_coda_root, 'annotation_validation')
json_cornercases = os.path.join(coda_root, 'corner_case.json')
original_labels = os.path.join(log_finn_root, 'original_labels_annotation')

In [5]:
if not os.path.exists(original_labels):
    os.makedirs(original_labels)
if not os.path.exists(lidar_labels_folder):
    os.makedirs(lidar_labels_folder)
if not os.path.exists(lidar_labels_annotation_folder):
    os.makedirs(lidar_labels_annotation_folder)

with open(annotation_validation, 'r') as f:
    anno_valid = f.read().splitlines()
    
with open(json_cornercases, 'r') as f:
    data_cornercases = json.load(f)

images = data_cornercases['images']
annotations = data_cornercases['annotations']

#### Functions to translate manual inspection into labels

Save points to coda_root in original format, and to original_labels in inference directory in KITTIodometry format

In [6]:
def save_points_image(points_kitti, points, image_id):
    points_kitti.astype('float32').tofile(os.path.join(original_labels, image_id + '.bin'))
    points.astype('float32').tofile(os.path.join(lidar_labels_folder, image_id + '.bin'))

def save_points_annotation(points_kitti, points, annotation_id):
    points_kitti.astype('float32').tofile(os.path.join(original_labels, annotation_id + '.bin'))
    points.astype('float32').tofile(os.path.join(lidar_labels_annotation_folder, annotation_id + '.bin'))

Rotate the point clouds

In [7]:
def rotate_nuscenes(points):
    new_points = np.zeros_like(points)
    new_points[:, 1] = -points[:, 0]
    new_points[:, 0] = points[:, 1]
    new_points[:, 2] = points[:, 2]
    
    return new_points[:, :3]
    
def rotate_once(points):
    new_points = np.zeros_like(points)
    new_points[:, 1] = points[:, 0]
    new_points[:, 0] = -points[:, 1]
    new_points[:, 2] = points[:, 2]
    
    return new_points[:, :3]

Load lidar data with labels and manual inspection clustering

In [13]:
def get_lidar_and_clustering(annotation):
    annotation_id = annotation['id']
    annotation_path = os.path.join(lidar_clustering_folder, str(annotation_id) + '.bin')
    clustering = anno_valid[annotation_id - 1]
    
    points = np.fromfile(annotation_path, dtype=np.float32).reshape((-1, 7))
     
    return points, clustering

Set labels according to manual inspection

In [9]:
def set_labels(points, clustering):
    labels = np.ones((points.shape[0], 1))
    if clustering == 1:
        # set dbscan
        labels[:, 0] = points[:, 4]
    elif clustering == 2:
        # set mean shift
        labels[:, 0] = points[:, 6]
    elif clustering == 3:
        # set dbscan
        labels[:, 0] = points[:, 4]
    elif clustering == 4:
        # set combined dbscan and mean shift
        for index, point in enumerate(points):
            if point[4] == 1 or point[6] == 1:
                labels[index, 0] = 1
            else:
                labels[index, 0] = -1
    elif clustering == 5:
        # set negated dbscan
        labels[:, 0] = -points[:, 4]
    elif clustering == 6:
        # keep all points as 1
        labels = labels
    elif clustering == 7:
        # keep all points as 1
        labels = labels
    elif clustering == 8:
        # set negated mean shift
        labels[:, 0] = -points[:, 6]
    elif clustering == 9:
        # keep all points as 1
        labels = labels
    return np.hstack([points[:, :3], labels])

Handle dublicates

In [10]:
def handle_dublicates(annotations_in_lidar):
    unique_rows, indices = np.unique(annotations_in_lidar, axis=0, return_index=True)
    # Filter all dublicates
    annotations_in_lidar_unique = annotations_in_lidar[np.sort(indices)]
    
    for index, annotation_1 in enumerate(annotations_in_lidar_unique):
        # if label already 1 skip
        if annotation_1[3] == 1:
            continue
        else:
            for annotation_2 in annotations_in_lidar:
                # if label is -1 in unique, check if there is a 1 label for this point in original array
                if annotation_1[0] == annotation_2[0] and annotation_1[1] == annotation_2[1] and annotation_1[2] == annotation_2[2] and annotation_2[3] == 1:
                    annotations_in_lidar_unique[index, 3] == 1
    return annotations_in_lidar_unique
    

#### Create labeled point clouds for each image and each annotation
Containing all points inside the 2D bounding boxes labeled as -1: no anomaly or 1: anomaly

Create labeled point clouds for each image (runtime: ~ 15min)

In [10]:
for image in tqdm(images):
    id = image['id']
    file_name = image['file_name'].split('.')[0]
    prefix, name = file_name.split('_')
    annotations_in_image = []
    annotations_in_lidar = np.array([-1,-1,-1,-1])
    
    for annotation in annotations:
        if(annotation['image_id']) == id:
            annotations_in_image.append(annotation)
    
    if prefix == 'kitti':
        for annotation in annotations_in_image:
            points, clustering = get_lidar_and_clustering(annotation)
            points = set_labels(points, int(clustering))
            annotations_in_lidar = np.vstack([annotations_in_lidar, points])
        annotations_in_lidar = handle_dublicates(annotations_in_lidar)
        annotations_in_lidar_kitti = np.copy(annotations_in_lidar)
        save_points_image(annotations_in_lidar_kitti, annotations_in_lidar, '{0:04d}'.format(int(id)))
        
    elif prefix == 'nuscenes':
        for annotation in annotations_in_image:
            points, clustering = get_lidar_and_clustering(annotation)
            points = set_labels(points, int(clustering))
            annotations_in_lidar = np.vstack([annotations_in_lidar, points])
        annotations_in_lidar = handle_dublicates(annotations_in_lidar)
        annotations_in_lidar_kitti = np.copy(annotations_in_lidar)
        annotations_in_lidar_kitti[:, :3] = rotate_nuscenes(annotations_in_lidar)
        save_points_image(annotations_in_lidar_kitti, annotations_in_lidar, '{0:04d}'.format(int(id)))
    else:
        for annotation in annotations_in_image:
            points, clustering = get_lidar_and_clustering(annotation)
            points = set_labels(points, int(clustering))
            annotations_in_lidar = np.vstack([annotations_in_lidar, points])
        annotations_in_lidar = handle_dublicates(annotations_in_lidar)
        annotations_in_lidar_kitti = np.copy(annotations_in_lidar)
        annotations_in_lidar_kitti[:, :3] = rotate_once(annotations_in_lidar)
        save_points_image(annotations_in_lidar_kitti, annotations_in_lidar, '{0:04d}'.format(int(id)))

100%|██████████| 1500/1500 [12:25<00:00,  2.01it/s]  


Create labeled point clouds for each annotation (runtime: ~ 20min)

In [ ]:
for annotation in tqdm(annotations):
    id = annotation['id']
    annotations_in_lidar = np.array([-1,-1,-1,-1])
    
    if id > 4413 and id <= 4812:
        points, clustering = get_lidar_and_clustering(annotation)
        points = set_labels(points, int(clustering))
        annotations_in_lidar = np.vstack([annotations_in_lidar, points])
        annotations_in_lidar_kitti = np.copy(annotations_in_lidar)
        save_points_annotation(annotations_in_lidar_kitti, annotations_in_lidar, '{0:04d}'.format(int(id)))
        
    elif id > 4812:
        points, clustering = get_lidar_and_clustering(annotation)
        points = set_labels(points, int(clustering))
        annotations_in_lidar = np.vstack([annotations_in_lidar, points])
        annotations_in_lidar_kitti = np.copy(annotations_in_lidar)
        annotations_in_lidar_kitti[:, :3] = rotate_nuscenes(annotations_in_lidar)
        save_points_annotation(annotations_in_lidar_kitti, annotations_in_lidar, '{0:04d}'.format(int(id)))
    elif id <= 4413:
        points, clustering = get_lidar_and_clustering(annotation)
        points = set_labels(points, int(clustering))
        annotations_in_lidar = np.vstack([annotations_in_lidar, points])
        annotations_in_lidar_kitti = np.copy(annotations_in_lidar)
        annotations_in_lidar_kitti[:, :3] = rotate_once(annotations_in_lidar)
        print(annotations_in_lidar_kitti)
        save_points_annotation(annotations_in_lidar_kitti, annotations_in_lidar, '{0:04d}'.format(int(id)))